In [1]:
import numpy as np
from quaos.core.paulis import PauliSum, PauliString
from quaos.core.circuits.target import find_map_to_target_pauli_sum
from quaos.core.circuits.gates import Gate
from quaos.utils import get_linear_dependencies
from quaos.graph_utils import plot_group_graph, find_swappable_pairs

First we make a random Pauli Sum, and obtain a list of the linearly independent rows

In [3]:
# parameters
n_paulis = 22
n_qudits = 10
d = 2
n_weights = 3 # number of different weights

dimensions = [d] * n_qudits
# make a random pauli sum
H = PauliSum.from_random(n_paulis, n_qudits, dimensions, rand_weights=False)

# this bit makes sections of H have different weights
weights = np.empty(n_paulis, dtype=int)
section_size = n_paulis // n_weights
for i in range(n_weights):
    start = i * section_size
    end = (i + 1) * section_size if i < n_weights - 1 else n_paulis
    weights[start:end] = i + 1

H.weights = weights
# print(H.symplectic())

# obtain linearly independent rows - note this fails sometimes as there is a solver in it that
# is not that robust. The Clifford approach is more robust as it creates a simple basis where the
# dependencies can be easily read.
independent_paulis, dependencies = get_linear_dependencies(H.symplectic(), d)

print(independent_paulis)
print(dependencies)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
{20: [(1, 1), (2, 1), (5, 1), (9, 1), (10, 1), (11, 1), (15, 1), (16, 1), (17, 1), (19, 1)], 21: [(1, 1), (5, 1), (6, 1), (7, 1), (9, 1), (10, 1), (13, 1), (14, 1), (15, 1), (16, 1)]}


We now solve for possible permutations which leave the dependencies unchanged

To do so we make a dictionary with labels of the coefficients

***Note this will only work for qubits for now, as for qudits we also need to account for the fact that a dependency can be made of $P_i + kP_j$ for some integer $k < d$***

In [4]:
cs = H.weights

graph_dict = {}

for i in independent_paulis:
    key = cs[i]
    if key in graph_dict:
        graph_dict[key].append([i])
    else:
        graph_dict[key] = [[i]]

for i in dependencies.keys():
    key = cs[i]
    dependency = dependencies[i]
    dependence_indices = [x[0] for x in dependency]
    dependence_multiplicities = [x[1] for x in dependency]  # this will be needed for qudits! always 1 for now
    if key in graph_dict:
        graph_dict[key].append(dependence_indices)
    else:
        graph_dict[key] = [dependence_indices]

print(graph_dict)



{np.int64(1): [[0], [1], [2], [3], [4], [5], [6]], np.int64(2): [[7], [8], [9], [10], [11], [12], [13]], np.int64(3): [[14], [15], [16], [17], [18], [19], [1, 2, 5, 9, 10, 11, 15, 16, 17, 19], [1, 5, 6, 7, 9, 10, 13, 14, 15, 16]]}


We then test to see if any indexes have an automorphism that can be found via a simple swap

***There is an additional condition - the pairs when swapped must leave the symplectic product matrix invariant...***
As there is always going to be a reasonably small number of pairs, we can probably check the combinatorial number of
options for this...

In [5]:
pairs = find_swappable_pairs(graph_dict)
print(pairs)

[(0, 3), (0, 4), (3, 4), (1, 5), (7, 13), (8, 12), (9, 10), (17, 19), (15, 16)]


Now we choose a target - for this example, performing all of these swaps

We then find the symplectic which maps the original H to H_target

In [9]:
# make target
H_target = H.copy()
for p in pairs:
    H_target.swap_paulis(p[0], p[1])

print(np.array_equal(H_target.symplectic_product_matrix(), H.symplectic_product_matrix()))

# # find F
# F, _, _, _ = find_map_to_target_pauli_sum(H, H_target)

# print(F)
# # check F
# print(np.array_equal(H_target.symplectic(), H.symplectic() @ F.T % d))



False
